In [1]:
import os
os.chdir("/Users/apple/Downloads/프로젝트1/ai06-level1-project")
os.listdir()

['test_images', 'train_annotations', '.DS_Store', 'train_images']

In [2]:
import os
import json
from collections import OrderedDict

ROOT = "/Users/apple/Downloads/프로젝트1/ai06-level1-project"
ANN_DIR = os.path.join(ROOT, "train_annotations")

category_ids = OrderedDict()

cnt_json = 0

for cur_root, dirs, files in os.walk(ANN_DIR):
    for fname in files:
        if not fname.endswith(".json"):
            continue
        fpath = os.path.join(cur_root, fname)
        cnt_json += 1

        try:
            with open(fpath, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print("JSON 로드 실패:", fpath, e)
            continue

        # COCO 형식 가정: "categories" 안에 id, name 있음
        if "categories" not in data:
            continue

        for cat in data["categories"]:
            cid = cat["id"]
            name = cat.get("name", str(cid))

            # 처음 본 id면 저장
            if cid not in category_ids:
                category_ids[cid] = name
            else:
                # 이름이 달라지면 경고만 출력
                if category_ids[cid] != name:
                    print("카테고리 이름 불일치 경고:", cid, category_ids[cid], "!=", name)

out_path = os.path.join(ROOT, "category_id_mapping.json")
with open(out_path, "w", encoding="utf-8") as f:
    # 키를 str로 바꿔서 저장
    json.dump({str(k): v for k, v in category_ids.items()},
              f, ensure_ascii=False, indent=2)

print("스캔한 json 개수:", cnt_json)
print("총 클래스 개수:", len(category_ids))
print("저장 완료:", out_path)


스캔한 json 개수: 1001
총 클래스 개수: 56
저장 완료: /Users/apple/Downloads/프로젝트1/ai06-level1-project/category_id_mapping.json


In [4]:
import os
import json
import random

ROOT = "/Users/apple/Downloads/프로젝트1/ai06-level1-project"
ANN_DIR = os.path.join(ROOT, "train_annotations")

all_jsons = []

# train_annotations 아래 모든 json 상대경로 수집
for cur_root, dirs, files in os.walk(ANN_DIR):
    for fname in files:
        if not fname.endswith(".json"):
            continue
        full_path = os.path.join(cur_root, fname)
        rel_path = os.path.relpath(full_path, ANN_DIR)  # 상대경로
        all_jsons.append(rel_path)

all_jsons.sort()
random.seed(42)
random.shuffle(all_jsons)

n_total = len(all_jsons)
n_train = int(n_total * 0.8)

train_files = all_jsons[:n_train]
val_files   = all_jsons[n_train:]

split_info = {
    "train": train_files,
    "val": val_files
}

out_path = os.path.join(ROOT, "split.json")
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(split_info, f, ensure_ascii=False, indent=2)

print("총 파일:", n_total)
print("train:", len(train_files), "val:", len(val_files))
print("저장 완료:", out_path)


총 파일: 1001
train: 800 val: 201
저장 완료: /Users/apple/Downloads/프로젝트1/ai06-level1-project/split.json
